In [ ]:
name = "Magnetics-Curve-Fit-Equation-Tool-2020.xlsx"
init = pd.read_excel(name, skiprows = 7, skipfooter=4).dropna(axis=1, how="all")
data = dict()
data["perm_dc"] = init[init.columns[0:7]]
data["core_loss"] = init[init.columns[7:13]]
data["normal_magn"] = init[init.columns[13:22]]
data["perm_freq"] = init[init.columns[22:31]]
data["perm_temp"] = init[init.columns[31:40]]

In [ ]:
for k, v in data.items():
    cols = v.columns
    new_cols = [x.split(".")[0].rstrip(":").lower() for x in cols]
    new_cols[1] = "perm"
    v.columns = new_cols
    v = v.fillna(method="ffill")
    try:
        index = v[v.perm == "58339, 58167, 58620, 58070, 58740, 58866, 58906, 58778, 58098, 58340:"].index[0]
        print("index {}".format(index))
    except IndexError:
        index = None
        print("index not found")
    if index:
        v.at[(index+1), "material"] = "High Flux Special"
        v.at[(index+2), "material"] = "High Flux Special"
        v.drop(axis = 0, labels = index, inplace = True)
    data[k] = v
    v.to_csv(f"{k}.csv")

In [ ]:
PERM_DC = pd.read_csv("perm_dc.csv")

def calculate_perm_dc(material, init_perm, drive_level):
    config = PERM_DC[PERM_DC.material == material][PERM_DC.perm == init_perm]
    a = float(config.a)
    b = float(config.b)
    c = float(config.c)
    if hasattr(drive_level, "__iter__"):
        last = 1 / (a + b * pow(drive_level[len(drive_level) - 1], c))
        if last <= 30:
            raise AttributeError("calculated value is out of range")
        # print("iterable")
        return [1 / (a + b * pow(x, c)) for x in drive_level]
    else:
        # print("not interable")
        calc= 1 / (a + b * pow(drive_level, c))
        if calc <= 30:
            raise AttributeError("calculated value is out of range")
        return 1 / (a + b * pow(drive_level, c))
calculate_perm_dc("MPP", 60, range(0, 100))